In [1]:
!nvidia-smi

Sat May 22 10:54:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%pip install -q tensorflow-addons[tensorflow]
%pip install -q simpleitk
# %pip install -q colab_ssh
%pip install -q wandb

     |████████████████████████████████| 686kB 15.9MB/s 
     |████████████████████████████████| 47.4MB 113kB/s 
     |████████████████████████████████| 1.8MB 15.4MB/s 
     |████████████████████████████████| 174kB 47.6MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 133kB 43.3MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 


In [3]:
!gdown https://drive.google.com/uc?id=1jYb-dKqywLm5bQ5esP6YSV5xuBzqBPqm

Downloading...
From: https://drive.google.com/uc?id=1jYb-dKqywLm5bQ5esP6YSV5xuBzqBPqm
To: /content/BRATS_Dataset.tar.gz
2.76GB [00:45, 60.3MB/s]


In [4]:
!apt -qq install pigz

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  pigz
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 57.4 kB of archives.
After this operation, 259 kB of additional disk space will be used.
Selecting previously unselected package pigz.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../archives/pigz_2.4-1_amd64.deb ...
Unpacking pigz (2.4-1) ...
Setting up pigz (2.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [5]:
!pigz -dc BRATS_Dataset.tar.gz | tar xf -

In [6]:
!rm BRATS_Dataset.tar.gz

In [7]:
%cd /content
!rm -rf 3D_res_attn_Unet

/content


In [8]:
!git clone https://github.com/ShivamShrirao/3D_res_attn_Unet

Cloning into '3D_res_attn_Unet'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 151 (delta 102), reused 94 (delta 45), pack-reused 0
Receiving objects: 100% (151/151), 546.24 KiB | 1.37 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [9]:
%cd 3D_res_attn_Unet

/content/3D_res_attn_Unet


In [10]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [11]:
import SimpleITK as sitk

In [12]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import glob
from tqdm.notebook import tqdm
from time import time
%matplotlib inline

In [13]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [14]:
def get_data_paths(dataset_path):
    t1 = glob.glob(dataset_path+'/*GG/*/*t1.nii.gz')
    t2 = glob.glob(dataset_path+'/*GG/*/*t2.nii.gz')
    t1ce = glob.glob(dataset_path+'/*GG/*/*t1ce.nii.gz')
    flair = glob.glob(dataset_path+'/*GG/*/*flair.nii.gz')
    seg = glob.glob(dataset_path+'/*GG/*/*seg.nii.gz')
    return list(zip(t1, t2, t1ce, flair, seg))

In [15]:
train_paths = get_data_paths('../BRATS_Dataset/brats_dataset/')
val_paths = get_data_paths('../BRATS_Dataset/brats_test_dataset/')

In [16]:
len(train_paths), len(val_paths)

(285, 50)

In [17]:
from model.preprocess import *

In [18]:
mean = tf.constant([-0.7050362 , -0.76629126, -0.8043347 , -0.77532315], shape=(1, 4, 1, 1, 1))
std  = tf.constant([0.40932664, 0.33403134, 0.26365048, 0.31701902], shape=(1, 4, 1, 1, 1))

In [19]:
# mean = 0
# std = 0
# for i, (img,lbl) in tqdm(enumerate(train_ds)):
#     if i == 200//BATCH_SIZE:
#         break
#     mean+= tf.reduce_mean(img, axis=[0,2,3,4])
#     std += tf.math.reduce_std(img, axis=[0,2,3,4])
# mean = tf.reshape(mean/i, (1, 4, 1, 1, 1))
# std = tf.reshape(std/i, (1, 4, 1, 1, 1))

In [20]:
depth = 128 # 155
height = 192 # 240
width = 160 # 240
(155-depth, 240-height, 240-width)

(27, 48, 80)

In [21]:
def random_crop3D(imgs, lbls, train=True):          # Crops image to size (128, 192, 160)
    d_cr, h_cr, w_cr = 13, 24, 40
    if train:
        if tf.random.uniform([], 0, 1, dtype=tf.float32) <= 0.8:
            d_cr = tf.random.uniform([], 0, 27, dtype=tf.int32)
            h_cr = tf.random.uniform([], 0, 48, dtype=tf.int32)
            w_cr = tf.random.uniform([], 0, 80, dtype=tf.int32)
    imgs = imgs[:, :, d_cr:d_cr+depth, h_cr:h_cr+height, w_cr:w_cr+width]
    lbls = lbls[:, :, d_cr:d_cr+depth, h_cr:h_cr+height, w_cr:w_cr+width]
    return imgs, lbls

In [22]:
def final_augmentation(imgs, lbls, train=True):       # input imgs[B, 4, 155, 240, 240], output[B, 4, 128, 192, 160]
    imgs, lbls = random_crop3D(imgs, lbls, train)
    if train:
        if tf.random.uniform([], 0, 1, dtype=tf.float32) <= 0.8:
            imgs = tf.image.random_brightness(imgs, 0.2)      # source code checked, it's fine for 3D
    # imgs = tf.image.per_image_standardization(imgs)   # source code checked, it's fine for 3D
    imgs = (imgs - mean) / std
    return imgs, lbls

In [23]:
BATCH_SIZE = 2
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [24]:
def get_tfds(data_paths, train=False):
    # 53.3373920917511 s
    dataset = tf.data.Dataset.from_tensor_slices(data_paths)
    if train:
        dataset = dataset.shuffle(len(data_paths))
    dataset = dataset.map(load_paths_wrapper, num_parallel_calls=AUTOTUNE)
    # if train:
    #     dataset = dataset.map(random_rotate3D, num_parallel_calls=AUTOTUNE)
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    if train:
        dataset = dataset.map(random_flip3D, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(lambda x,y: final_augmentation(x,y,train), num_parallel_calls=AUTOTUNE)      # remove for test,val
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [25]:
train_ds = get_tfds(train_paths, train=True)
val_ds = get_tfds(val_paths)

In [26]:
import wandb
from wandb.keras import WandbCallback

In [27]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [210]:
config_defaults = {
    'activation'            : 'gelu', # selu, elu, swish
    'batch_size'            : BATCH_SIZE,
    'block_type'            : 'AttnBottleneckBlock',
    'clr_step_size'         : 10,
    'clr_offset_ep'         : 7,
    'groups'                : 32,
    'dp_rate'               : 0.22,
    'dropout_type'          : 'Spatial',
    'frac_dv'               : 1,
    'loss_function'         : 'FocalTversky',
    'max_lr'                : 6e-4,
    'min_lr'                : 1e-6,
    'nheads'                : 8,
    'norm'                  : 'gn',
    'optimizer'             : 'adamw',
    'vary_weight_decay'     : True,
    'weight_decay'          : 1e-4,     # max weight decay
    'z_notes'               : ['removed random rotation', '-1 to 1 imgs']
}
CONFIG = config_defaults

In [29]:
# run = wandb.init(id='vp3j4jsm', project="3D_res_attn_Unet", resume='must')
# CONFIG = run.config

wandb: Currently logged in as: shivamshrirao (use `wandb login --relogin` to force relogin)


In [ ]:
run = wandb.init(project="3D_res_attn_Unet", entity="shivamshrirao", config=config_defaults)
CONFIG = wandb.config

wandb: Currently logged in as: shivamshrirao (use `wandb login --relogin` to force relogin)


In [ ]:
mean.numpy().ravel(), std.numpy().ravel()

(array([-0.7050362 , -0.76629126, -0.8043347 , -0.77532315], dtype=float32),
 array([0.40932664, 0.33403134, 0.26365048, 0.31701902], dtype=float32))

In [30]:
tf.keras.backend.set_image_data_format('channels_first')

In [31]:
## attention shortcuts to be used.
## try attention on the unet shortcuts
## try Global Context Block
## try input image pyramid and deep supervised output layers (output from intermediate layers (fuse with conv-sigmoid like U^2net ??))
## TRY PIXELSHUFFLE
## Try SPARSE CONV NET https://github.com/facebookresearch/SparseConvNet

In [32]:
from model.conv_blocks import *

In [33]:
def enc_dec(x, block, frac_dv, stack_args):     # x(64,64,96,80)
    x1 = down_stack(x ,  64, nblocks=2, block=block, strides=2, **stack_args)     # (256,32,48,40)
    x2 = down_stack(x1, 128, nblocks=2, block=block, strides=2, **stack_args)     # (512,16,24,20)
    x3 = down_stack(x2, 256, nblocks=2, block=block, strides=2, frac_dv=frac_dv, **stack_args) # (1024, 8,12,10)
    x4 = down_stack(x3, 256, nblocks=3, block=block, strides=2, frac_dv=frac_dv, **stack_args) # (1024, 4, 6, 5)

    y = up_stack(x4,x3, 128, nblocks=3, block=block, strides=2, frac_dv=frac_dv, **stack_args) # (1024+1024->512,8,12,10)
    y = up_stack(y, x2,  64, nblocks=2, block=block, strides=2, **stack_args)     # (512+512->256,16,24,20)
    y = up_stack(y, x1,  32, nblocks=2, block=block, strides=2, **stack_args)     # (256+256->128,32,48,40)
    y = up_stack(y, x ,  16, nblocks=1, block=block, strides=2, **stack_args)     # (128+ 64-> 64,64,96,80)
    return y


def build_network(cfg, block, input_shape=(4,128,192,160), classes=4):
    stack_args = {'activation': cfg['activation'], 'groups': cfg['groups'], 'norm': cfg['norm'],
                  'dp_rate': cfg['dp_rate'], 'dropout_type': cfg['dropout_type']}

    inp = layers.Input(shape=input_shape)                           # ( 4,128,192,160)
    x = inp
    x = ConvNorm(32, kernel_size=3, strides=2, activation=cfg['activation'], norm=cfg['norm'])(x)   # (32,64,96,80)
    x = ConvNorm(64, kernel_size=3, do_norm_act=False)(x)                               # (64, 64, 96, 80)

    x = enc_dec(x, block, cfg['frac_dv'], stack_args)                                   # (64, 64, 96, 80)

    x = NormAct(activation=cfg['activation'], norm=cfg['norm'])(x)
    x = layers.UpSampling3D(data_format="channels_first")(x)                            # (64,128,192,160)
    x = ConvNorm(16, kernel_size=3, activation=cfg['activation'], norm=cfg['norm'])(x)  # (16,128,192,160)
    x = ConvNorm(classes, kernel_size=3, do_norm_act=False)(x)                          #(3,128,192,160)
    x = layers.Softmax(axis=1)(x)         # softmax cause each pixel has unique class, no overlap with other classes, verified.

    return tf.keras.Model(inputs=inp, outputs=x)

In [ ]:
block = AttnBottleneckBlock
CONFIG['block_type'] = block.__name__
model = build_network(CONFIG, block, input_shape=(4,depth,height,width), classes=4)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4, 128, 192, 0                                            
__________________________________________________________________________________________________
conv_norm (ConvNorm)            (None, 32, 64, 96, 8 3520        input_1[0][0]                    
__________________________________________________________________________________________________
conv_norm_1 (ConvNorm)          (None, 64, 64, 96, 8 55360       conv_norm[0][0]                  
__________________________________________________________________________________________________
attn_bottleneck_block (AttnBott (None, 256, 32, 48,  43344       conv_norm_1[0][0]                
______________________________________________________________________________________________

In [ ]:
model_plot = tf.keras.utils.plot_model(model, '/content/model.png', show_shapes=True, dpi=None)

In [ ]:
wandb.log({"model": wandb.Image("/content/model.png")})

In [106]:
from tensorflow.python.client import session
from tensorflow.python.summary import summary

In [253]:
run.dir

'/content/3D_res_attn_Unet/wandb/run-20210522_105710-vp3j4jsm/files'

In [254]:
log_dir = run.dir+"/trace_log"
!rm -rf $log_dir

@tf.function
def trace(x):
    return model(x)

data = tf.random.normal((2,) + model.input_shape[1:])
data = tf.constant(data)

imported_g = trace.get_concrete_function(data).graph

# Export the graph
with session.Session(graph=imported_g) as sess:
    pb_visual_writer = summary.FileWriter(log_dir)
    pb_visual_writer.add_graph(sess.graph)

In [ ]:
%tensorboard --logdir /content/trace_log

In [109]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(6006)"))

https://00u60km71adg-496ff2e9c6d22116-6006-colab.googleusercontent.com/


In [34]:
import gc
gc.collect()

1050

In [35]:
from model.losses import *
from model.utils import *

In [258]:
steps_per_epoch = int(np.ceil(len(train_paths)/BATCH_SIZE))
clr_offset = CONFIG["clr_offset_ep"]*steps_per_epoch
clr = CustomCLR(
        offset=clr_offset,
        initial_learning_rate=CONFIG["min_lr"],
        maximal_learning_rate=CONFIG["max_lr"],
        step_size=CONFIG["clr_step_size"] * steps_per_epoch,
        scale_mode="cycle")

In [237]:
wdc = CustomCLR(
        offset=clr_offset,
        initial_learning_rate=CONFIG["weight_decay"]*CONFIG["min_lr"]/CONFIG["max_lr"],
        maximal_learning_rate=CONFIG["weight_decay"],
        step_size=CONFIG["clr_step_size"] * steps_per_epoch,
        scale_mode="cycle")

In [239]:
if CONFIG["optimizer"] == 'adamw':
    opt = tfa.optimizers.AdamW(learning_rate=clr, weight_decay=wdc)
elif CONFIG["optimizer"] == 'adam':            # or just make weight_decay=0
    opt = tf.keras.optimizers.Adam(learning_rate=clr)
opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)

In [240]:
if CONFIG["loss_function"] == 'FocalTversky':
    loss_function = FocalTversky()
elif CONFIG["loss_function"] == 'cce':
    loss_function = cce

In [241]:
model.compile(optimizer=opt, loss=loss_function, metrics=[dsc])

In [251]:
model.save(wandb.run.dir+"/model-best.h5")

In [252]:
wandb.save("*.h5")

['/content/3D_res_attn_Unet/wandb/run-20210522_105710-vp3j4jsm/files/model-best.h5']

In [243]:
# model = tf.keras.models.load_model(wandb.restore("model-best.h5").name, custom_objects={'ConvNorm': ConvNorm,
#                                                                                         'NormAct': NormAct,
#                                                                                         'AttnBottleneckBlock': AttnBottleneckBlock,
#                                                                                         'BasicBlock': BasicBlock,
#                                                                                         'InvertedResBlock': InvertedResBlock,
#                                                                                         'SqueezeExcite': SqueezeExcite,
#                                                                                         'MHSA3D': MHSA3D,
#                                                                                         'AbsPosEmb': AbsPosEmb,
#                                                                                         'dsc': dsc,
#                                                                                         'FocalTversky': FocalTversky,
#                                                                                         'CustomCLR': CustomCLR,
#                                                                                         })
# model.compile(optimizer=model.optimizer, loss=model.loss, metrics=[dsc])

In [261]:
initial_epoch = 40
model.optimizer.iterations//steps_per_epoch

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [41]:
wandb_cb = WandbCallback(monitor='dsc', mode='max')

In [42]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        img, lbl = next(iter(val_ds))
        pred = self.model(img)
        i = epoch % BATCH_SIZE
        img = img[i,1].numpy()
        mn = img.min()
        mx = img.max()
        img = (img - mn)/(mx - mn) * 255
        lbl = lbl[i,1:].numpy()*255
        pred = pred[i,1:].numpy()*255
        get_gif(img, lbl, pred, "out.gif")
        wandb.log({"Outputs": wandb.Image("out.gif")})
        gc.collect()

In [ ]:
history = model.fit(train_ds,
                    epochs=100,
                    steps_per_epoch=np.ceil(len(train_paths)/BATCH_SIZE),
                    validation_data=val_ds,
                    validation_steps=np.ceil(len(val_paths)//BATCH_SIZE),
                    initial_epoch=initial_epoch,
                    workers=4,
                    use_multiprocessing=True,
                    callbacks=[CustomCallback(), wandb_cb])

In [247]:
model.evaluate(val_ds, steps=np.ceil(len(val_paths)//BATCH_SIZE))

25/25 [==============================] - 44s 1s/step - loss: 1.3295 - dsc: 0.6339


[1.3140285015106201, 0.6369562745094299]

In [ ]:
scores = []
for i, (a,b) in tqdm(enumerate(train_ds)):
    if i==steps_per_epoch:
        break
    o = model(a)
    o = tf.cast(o, tf.float32)
    scores.append(dsc(b,o))

In [ ]:
np.mean(scores, axis=0)

In [ ]:
run.finish()